In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

external_csv = []
openstack_csv = []
import os
for dirname, _, filenames in os.walk(os.getcwd()):
    for filename in filenames:
        if 'external' in filename:
            fx = os.path.join(dirname, filename)
            external_csv.append(fx)            
            print(fx)
        elif 'internal' in filename:
            fx = os.path.join(dirname, filename)
            openstack_csv.append(fx)
            print(fx)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

c:\Users\cupca\Desktop\Thesis\thesis-project\WISENT-CIDDS-001\CIDDS-001\traffic\ExternalServer\CIDDS-001-external-week1.csv
c:\Users\cupca\Desktop\Thesis\thesis-project\WISENT-CIDDS-001\CIDDS-001\traffic\ExternalServer\CIDDS-001-external-week2.csv
c:\Users\cupca\Desktop\Thesis\thesis-project\WISENT-CIDDS-001\CIDDS-001\traffic\ExternalServer\CIDDS-001-external-week3.csv
c:\Users\cupca\Desktop\Thesis\thesis-project\WISENT-CIDDS-001\CIDDS-001\traffic\ExternalServer\CIDDS-001-external-week4.csv
c:\Users\cupca\Desktop\Thesis\thesis-project\WISENT-CIDDS-001\CIDDS-001\traffic\OpenStack\CIDDS-001-internal-week1.csv
c:\Users\cupca\Desktop\Thesis\thesis-project\WISENT-CIDDS-001\CIDDS-001\traffic\OpenStack\CIDDS-001-internal-week2.csv
c:\Users\cupca\Desktop\Thesis\thesis-project\WISENT-CIDDS-001\CIDDS-001\traffic\OpenStack\CIDDS-001-internal-week3.csv
c:\Users\cupca\Desktop\Thesis\thesis-project\WISENT-CIDDS-001\CIDDS-001\traffic\OpenStack\CIDDS-001-internal-week4.csv


# CIDDS-001 Cleaning
This notebook presents the cleanup done on CIDDS-001, an academic intrusion detection dataset published by HS-Coburg. I have the follow-up CIDDS-002 here as well, preprocessed in the same way, to allow usage together in a bigger analysis.

In [2]:
preproc = 'externalserver'
if preproc == 'externalserver':
    csvs = external_csv
elif preproc == 'openstack':
    csvs = openstack_csv

## Just reading the data
It is distributed in multiple CSV files, separated in the 2 tracks of the experiment OpenStack, the controlled environment & ExternalServer.

In [3]:
df = pd.concat(objs=[pd.read_csv(fp, encoding='utf-8') for fp in csvs], ignore_index=True, copy=False)

C:\Users\cupca\AppData\Local\Temp\ipykernel_16560\3869627319.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.concat(objs=[pd.read_csv(fp, encoding='utf-8') for fp in csvs], ignore_index=True, copy=False)


## Some renaming

In [4]:
df = df.rename(str.lower, axis='columns')
df = df.rename(str.strip, axis='columns')
df.rename(columns={
            'date first seen': 'date_first_seen', 
            'src ip addr': 'src_ip_addr',
            'src pt': 'src_pt',
            'dst ip addr': 'dst_ip_addr',
            'dst pt': 'dst_pt',
            'attacktype': 'attack_type',
            'attackid': 'attack_id',
            'attackdescription': 'attack_description',
            'class': 'label'
        }, inplace=True)

In [5]:
df.columns

Index(['date_first_seen', 'duration', 'proto', 'src_ip_addr', 'src_pt',
       'dst_ip_addr', 'dst_pt', 'packets', 'bytes', 'flows', 'flags', 'tos',
       'label', 'attack_type', 'attack_id', 'attack_description'],
      dtype='object')

In [1]:
df.head(100)

NameError: name 'df' is not defined

: 

## Dropping metadata
Some columns are not intended for use because they are metadata. Flow ID, IP addresses, timestamps, etc.

If you are unsure why source and destination port are also removed you can read [Establishing the Contaminating Effect of Metadata Feature Inclusion in Machine-Learned Network Intrusion Detection Models](https://link.springer.com/chapter/10.1007/978-3-031-09484-2_2). In short: any included metadata feature will act as a (very) powerful shortcut predictor.

In [7]:
df = df.drop(labels=['date_first_seen', 'src_ip_addr', 'src_pt', 'dst_ip_addr', 'dst_pt', 'attack_description'], axis=1)        

## Solving a weird encoding of values > 1e7.

In [8]:
def remove_million(val):
    if type(val) is str:                
        val = val.strip()
        if ' M' in val:                    
            val = val.replace('.', '')
            val = val.replace(' M', '00000')                    
        return val
    elif type(val) is int:
        return val

In [9]:
df['bytes'] = df['bytes'].apply(remove_million)
df['bytes'] = pd.to_numeric(df['bytes'], errors='raise', downcast='float')

In [11]:
df.columns

Index(['duration', 'proto', 'packets', 'bytes', 'flows', 'flags', 'tos',
       'label', 'attack_type', 'attack_id'],
      dtype='object')

## Transforming the custom flag encoding into a one-hot representation

In [13]:
print("Flags find 0xdb")
print("0xdb", df[df['flags'] == '0xdb'].index)        
print("Custom one-hot encode for the flags")
df['flags'] = df['flags'].str.strip()
flags_idx = df.columns.get_loc('flags')
ohe_flag_data = pd.DataFrame(
      data= df['flags'].apply(func=lambda flag_str: [0 if c == '.' else 1 for c in flag_str]).to_list(),
      columns=['tcp_urg', 'tcp_ack', 'tcp_psh', 'tcp_rst', 'tcp_syn', 'tcp_fin'],
      dtype=np.int8)
for i, c in enumerate(ohe_flag_data.columns):
    df.insert(loc=flags_idx+i, column=c, value=ohe_flag_data[c])
df = df.drop(labels=['flags'], axis=1)

Flags find 0xdb
0xdb Index([], dtype='int64')
Custom one-hot encode for the flags


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
df['flags'].sum()

In [ ]:
df['attack_type'] = df['attack_type'].replace({'---': 'benign'})

In [ ]:
df['attack_id'] = df['attack_id'].replace({'---': 0})
df['attack_id'] = df['attack_id'].astype(np.int32)

## Downsizing
The individual frames are optimized by downsizing their types to more appropriate forms than the default float64, int64 or object (str) types.

In [ ]:
from fastai.tabular.all import df_shrink

In [ ]:
df.dtypes

In [ ]:
df = df_shrink(df)

In [ ]:
df.dtypes

## Removing infinity and NaN

In [10]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [11]:
df.shape

(671241, 10)

In [12]:
print("N/A rows after preproc", df.isna().any(axis=1).sum())
df.dropna(inplace=True)
print(df.shape)

N/A rows after preproc 0
(671241, 10)


## Dropping Duplicates
There should be no duplicates because they can bias training and can lead to over-optimistic estimates of classification performance during testing.

In [ ]:
dupli = df.duplicated().sum()
if dupli > 0:
    print(dupli, "fully duplicate rows")
    df.drop_duplicates(inplace=True)
df.shape

In [ ]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df.to_parquet(f'cidds-001-{preproc}.parquet')